#### Start

In [ ]:
# mount
from google.colab import drive
drive.mount('/content/drive')

In [4]:
import os
# Load token from file
with open('/content/drive/MyDrive/agri-llm/hf_token.txt', 'r') as file:
    hf_token = file.read().strip()
os.environ["HUGGINGFACEHUB_API_TOKEN"] = hf_token

In [ ]:
!pip install python-dotenv
!pip install -q -U langchain huggingface_hub sentence_transformers faiss-gpu
!pip install -U langchain-community
!pip install transformers datasets torch scikit-learn numpy

In [9]:
os.chdir('/content/drive/MyDrive/agri-llm')
from models.rag import config_setup as con
from models.rag import fn_rag as rag

#### RAG Main

In [ ]:
# Setup directories
con.setup_directory_structure()
con.setup_environment()

# Initialize RAG system
rag = rag.RAGSystem(
    documents_path=str(Config.DATA_DIR),
)

# Process documents
rag.load_documents()
rag.create_vector_store()
rag.setup_qa_chain()

# Save vector store
rag.save_vector_store(str(Config.VECTOR_STORE_DIR))

print("RAG system setup complete!")

# Interactive query loop
while True:
    question = input("\nEnter your question (or 'quit' to exit): ")
    if question.lower() == 'quit':
        break

    result = rag.query(question)
    print("\nAnswer:", result['answer'])
    print("\nSources used:")
    for i, doc in enumerate(result['source_documents'], 1):
        print(f"\nSource {i}:")
        print(doc[:200] + "..." if len(doc) > 200 else doc)